<a href="https://colab.research.google.com/github/mmaruthi/Deep_Learning_EVA4_Phase1/blob/master/S8_ResNet_Modular/Master_Model_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mounting the drive and copying the code from github to drive
from google.colab import drive 
drive.mount('/content/gdrive')
%cd gdrive/My Drive/project_folder
! ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/project_folder
Deep_Learning_EVA4_Phase1


In [2]:
!git clone https://github.com/mmaruthi/Deep_Learning_EVA4_Phase1

fatal: destination path 'Deep_Learning_EVA4_Phase1' already exists and is not an empty directory.


In [3]:
%cd Deep_Learning_EVA4_Phase1/S8_ResNet_Modular
! ls

/content/gdrive/My Drive/project_folder/Deep_Learning_EVA4_Phase1/S8_ResNet_Modular
data  Master_Model.ipynb  model  __pycache__  README.md  s8_resnet.py


In [0]:
from __future__ import print_function
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from model import Datatransforms,Model,Dataloaders,Train_Losses,Test_losses

In [9]:
data_loader = Dataloaders.DataLoader(shuffle=True,batch_size=128,seed=1)

mean = (0.49139968, 0.48215841, 0.44653091)
stdev = (0.24703223, 0.24348513, 0.26158784)
transform_instance  = Datatransforms.Transforms_custom(normalize=True, mean=mean, stdev=stdev)

fillmeans = (np.array(mean)*255).astype(np.uint8)
before_norm_list = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
               transforms.RandomRotation((-7,7),fill=(1,))]
               
test_transforms   = transform_instance.test_transforms()
train_transforms  = transform_instance.train_transforms(before_norm = None)

train_data = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=train_transforms)
test_data  = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=test_transforms)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [0]:
trainloader = data_loader.load(train_data)
testloader  = data_loader.load(test_data)

In [17]:
import s8_resnet
import torch
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
model = s8_resnet.ResNet18()
model.to(device)
summary(model, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1,

In [0]:
train_loss = Train_Losses.Train_loss()
test_loss  = Test_losses.Test_loss()

# Train and Test the model 

In [21]:
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
EPOCH     = 40
L2_factor = 0
L1_factor = 0.001
optimizer = optim.SGD(model.parameters(), lr=0.025, momentum=0.9,weight_decay=L2_factor)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(trainloader), epochs=EPOCH)
train_loss_all = []
train_acc_all  = []
test_loss_all  = []
test_acc_all   = []

for epoch in range(EPOCH):

    print("EPOCH:", epoch)
    train_losses, train_acc = train_loss.train_loss_calc(model,device,train_loader=trainloader,optimizer=optimizer,epoch=EPOCH,factor=L1_factor)
    test_losses, test_acc   = test_loss.test_loss_calc(model,device,test_loader=testloader)

    train_loss_all.extend(train_losses)
    train_acc_all.extend(train_acc)
    test_loss_all.extend(test_losses)
    test_acc_all.extend(test_acc)

  0%|          | 0/391 [00:00<?, ?it/s]

EPOCH: 0


Train Loss = 0.07020482420921326 Batch Id = 390 Train Accuracy = 98.47: 100%|██████████| 391/391 [00:58<00:00,  7.30it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.7156, Test Accuracy: 82.08

EPOCH: 1


Train Loss = 0.16180196404457092 Batch Id = 390 Train Accuracy = 97.99: 100%|██████████| 391/391 [00:59<00:00,  7.24it/s]
  0%|          | 0/391 [00:00<?, ?it/s]

...


Test set: Average loss: 0.6120, Test Accuracy: 87.19

EPOCH: 38


Train Loss = 0.005023177247494459 Batch Id = 390 Train Accuracy = 100.00: 100%|██████████| 391/391 [00:59<00:00,  6.59it/s]
  0%|          | 0/391 [00:00<?, ?it/s]


Test set: Average loss: 0.6127, Test Accuracy: 87.28

EPOCH: 39


Train Loss = 0.004941138438880444 Batch Id = 390 Train Accuracy = 100.00: 100%|██████████| 391/391 [00:59<00:00,  7.44it/s]



Test set: Average loss: 0.6105, Test Accuracy: 87.15

